## Test prompt for S0

In [ ]:
from constants.abs_paths import AbsDirPath

from processing_pipeline.utilities.data_transformation import load_all_files

In [ ]:

df_o = load_all_files(AbsDirPath.O_KEYWORDS_MATCHING)
df_o.shape

In [ ]:
df_o.head()

In [ ]:
df_o['nwords'] = df_o.sentence.str.count(" ") + 1
df_o = df_o[df_o.nwords > 5]


In [ ]:
df_o.groupby('sentence').nsimilar.agg("sum").sort_values(ascending=False)

In [ ]:
df_sample = df_o.groupby('sentence').nsimilar.agg("sum").sort_values(ascending=False).head(200).reset_index().drop(columns="nsimilar")

In [ ]:
AbsDirPath.PT_O_KEYWORDS_MATCHING.mkdir(exist_ok=True)
df_sample.to_parquet(AbsDirPath.PT_O_KEYWORDS_MATCHING / "sample.parquet", index=False, engine="pyarrow", compression="snappy")

In [ ]:
from processing_pipeline.s0_noise_filtering.NoiseFiltering import NoiseFilteringStage
from cfg.LLMHost import LLMHost

NoiseFilteringStage(hostname=LLMHost.GREEN_LAB, in_dir_override=AbsDirPath.PT_O_KEYWORDS_MATCHING, out_dir_override=AbsDirPath.PT_S0_NOISE_FILTERING, batch_size_override=20, disable_cache=True).execute()


In [ ]:
out_dir = AbsDirPath.PT_S0_NOISE_FILTERING
df_results = load_all_files(out_dir)
df_results.to_csv(out_dir / "sample.csv")

In [ ]:
df_results.head()

## Test prompt for S1

In [ ]:
import pandas as pd

df_merged = pd.merge(df_o, df_results.drop(columns=["s0_prompt"])[df_results.s0_to_eliminate == False], on="sentence", how="right")
df_merged.shape

In [ ]:
df_merged.head()

In [ ]:
df_merged = df_merged.drop_duplicates(["qa", "sentence"])
df_merged.shape